# Kernal Based

In [42]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.pyplot
import cvxopt
from cvxopt import solvers,matrix
from sklearn.svm import SVC

def lin_Data(size, feature):
    w0 = np.zeros(1)
    w = 0.5 * np.ones(shape=(feature,1))
    std_dev = 0.5
    data = []
    ex = size/2
    np.random.seed(4736)
# Saperable DataSet
    for i in range(0,size):
        (data.append (np.asarray(np.random.uniform(low=-1,high=1,size=feature),dtype='float32')))

# Non-Saperable DataSet
#   data = np.append((np.random.randn((size/2),feature)+std_dev),(np.random.randn((size/2),feature)-std_dev),axis =0)
    
    data = np.array(data)
    print data.shape
    np.random.shuffle(data)
    clabel = []
# Class value indentification
    for i in range(0,data.shape[0]):
        if(w0 + np.dot(w.T,data[i].T))> 0:
            clabel.append([1.0])
        else:
            clabel.append([-1.0])
    clabel = np.array(clabel)
#  Cross validation 
    Kfold = data.shape[0]/10
    testdata = data[-Kfold:,:]
    data = data[0:-Kfold,:]
    test_lab = clabel[-Kfold:,:]
    clabel = clabel[0:-Kfold,:]
    
#  Graph Plotting
    xp = []
    xn = []
    xpo = []
    ypo = []
    xne = []
    yne = []
    for i in range(0,clabel.shape[0]):
        if clabel[i]>0:
            xp.append(data[i])
        else:
            xn.append(data[i])
    for j in range(0,len(xp)-1):
        xpo.append(xp[j][0])
        ypo.append(xp[j][1])
    for k in range(0,len(xn)-1):
        xne.append(xn[k][0])
        yne.append(xn[k][1])
    plt.plot(xne,yne,'ro')
    plt.plot(xpo,ypo,'bo')
    plt.title("Dataset")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    return data,clabel,testdata,test_lab
size = 200
feature = 2
data, clabel,testdata,test_lab = lin_Data(size,feature)


(200, 2)


In [43]:
def kernel_comp(x,y,degree):
    
# Gaussian Kernel    
    gamma = 0.1
    k = np.exp(-1/gamma*(np.dot((x-y),((x-y).T))))
    
#     Polynomial Kernel
#     k= ((np.dot(x,y.T )+1)**2)
    return k
    
      
def alpha_solver(x, y):
    W = 0
    W0 = 0
    degree = 2
    k = kernel_comp(np.array(x),np.array(x),degree)
    
    var,xfeature = x.shape
    p = matrix(np.ndarray.tolist(np.dot(y.T,y) * k))
    q = cvxopt.base.matrix([-1.0] * var)
    G0 = np.zeros((var, var), float)
    np.fill_diagonal(G0, -1.0)
    G1 = np.zeros((var, var), float)
    np.fill_diagonal(G1, 1.0)
    g = matrix(np.vstack((G0,G1)))
    h0 = matrix([0.0] * var)
    h1 = matrix([0.5]*var)
    h= matrix(np.vstack((h0,h1)))
    a = matrix(np.ndarray.tolist(y))
    b = matrix(0.0)
    Ld = solvers.qp(p,q,g,h,a,b)['x']
    alpha = matrix(Ld)

    #print alpha_val
    support_vector = []
    for i in range(len(alpha)):
        if alpha[i] > 0:
            support_vector.append(x[i])
    support_vector = np.matrix(support_vector)
    #print len(support_vector1)
    for j in range(var):
        W += (alpha[j] * y[j] * x[j])

    for k in range(len(support_vector)):
        W0 += (y[k] - np.dot(W,x[k]))
    #print W, W0
    W0 = W0 / len(support_vector)
    
    return W0, W, np.array(support_vector)
    
w0, w, s_vector = alpha_solver(data,clabel)


     pcost       dcost       gap    pres   dres
 0: -2.0836e-03 -9.0006e+01  9e+01  3e-17  3e-16
 1: -2.7158e-03 -9.0367e-01  9e-01  3e-16  1e-15
 2: -2.7777e-03 -1.1808e-02  9e-03  2e-16  4e-16
 3: -2.7778e-03 -2.8681e-03  9e-05  2e-16  4e-16
 4: -2.7778e-03 -2.7787e-03  9e-07  2e-16  3e-16
 5: -2.7778e-03 -2.7778e-03  9e-09  2e-16  3e-16
Optimal solution found.


In [44]:
def predict_class(w0,w,data,clabel,testdata,test_lab,s_vector):
    predict_label = []
    true = 0
    slack = 0.978
#     Class value prediction
    for i in range(0,testdata.shape[0]):
        if(w0 + np.dot(w.T,testdata[i].T))> 0:
            predict_label.append([1.0])
        else:
            predict_label.append([-1.0])
    predict_label = np.array(predict_label)
#     print predict_label, test_lab
    for i in range(0,predict_label.shape[0]):
        if (predict_label[i]==test_lab[i]):
            true = true+1
    accuracy = float(true)/len(test_lab)
    xp = []
    xn = []
    xpo = []
    ypo = []
    xne = []
    yne = []
    
    for i in range(0,predict_label.shape[0]):
        if predict_label[i]>0:
            xp.append(testdata[i])
        else:
            xn.append(testdata[i])
    
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.2),
                         np.arange(y_min, y_max, 0.2))
#     Polynomial Kernel
#     clf = SVC(kernel='poly',degree=1)

#  Gaussian Kernel
    clf = SVC(kernel='rbf')

    
    for j in range(0,len(xp)-1):
        xpo.append(xp[j][0])
        ypo.append(xp[j][1])
    for k in range(0,len(xn)-1):
        xne.append(xn[k][0])
        yne.append(xn[k][1])
    spo = []
    sne= []
    
    for s in range(0,len(s_vector)-1):
        spo.append(s_vector[s][0])
        sne.append(s_vector[s][1])
#  Result Graph Plot     
    clf.fit(testdata,test_lab.ravel())
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)
    plt.plot(xne,yne,'ro')
    plt.plot(xpo,ypo,'bo')
    plt.scatter(spo,sne,c='yellow',marker='D',s=20)

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Gaussian Kernel Output")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    

    return predict_label

predict_label = predict_class(w0,w,data,clabel,testdata,test_lab,s_vector)
    

In [45]:
def confusion_matrix(testclass,ypredict):
    
    temp = [-1,1]
    cm= []
    for i in temp:
        tmp =[0]*len(temp)
        for j in range(len(testclass)):
            if testclass[j] == i and testclass[j] == ypredict[j]:
                tmp[temp.index(i)] += 1
            elif testclass[j] == i and testclass[j] != ypredict[j]:
                tmp[temp.index(ypredict[j])] += 1
        cm.append(tmp)
    cm = np.array(cm)
    print cm
    return cm

    
cm = confusion_matrix(test_lab,predict_label)

[[ 8  0]
 [ 0 12]]


In [46]:
def precisionCall(cm,testclass):
    precesion = np.zeros(2)
    recall = np.zeros(2)
    f1measure = np.zeros(2)
    accuracy = 0
    for i in range(0,2):
        for j in range(0,2):
            precesion[i] += cm[j][i]
            recall[i] += cm[i][j]
            if(i==j):
                accuracy = accuracy + cm[i][j]
        precesion[i] = cm[i][i]/precesion[i]
        recall[i] = cm[i][i]/recall[i]
        f1measure[i] = ((2*precesion[i]*recall[i])/(precesion[i]+recall[i]))
    accuracy = float(accuracy)/testclass.shape[0]
    print " Confusion Matrix:"+ str(cm[0])
    print "\t\t  "+ str(cm[1])
    print "         Precesion:   Recall:    F-1 Measures"
    print "Class-1: " + str(round(precesion[0],3))+"       "+str(round(recall[0],3))+"         "+str(round(f1measure[0],3))
    print "Class 1: " + str(round(precesion[1],3))+"        "+str(round(recall[1],3))+"        "+str(round(f1measure[1],3))
    print "\nAccuracy: "+str(round(accuracy,3)*100)+"%"
    return accuracy,precesion,recall,f1measure


    
accuracy,precesion,recall,f1measure = precisionCall(cm,test_lab)

 Confusion Matrix:[8 0]
		  [ 0 12]
         Precesion:   Recall:    F-1 Measures
Class-1: 1.0       1.0         1.0
Class 1: 1.0        1.0        1.0

Accuracy: 100.0%
